In [53]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

PATH = '/Users/soyoung/ml_project/airplane/train.csv'
data = pd.read_csv(PATH)
df = pd.DataFrame(data)
encoder = LabelEncoder()

df.replace('Delayed',1,inplace=True)
df.replace('Not_Delayed',0,inplace=True)
df.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN


In [45]:
df['date'] = df['Month'].astype(str)+'-'+df['Day_of_Month'].astype(str)
df['date'] = pd.to_datetime(df['date'], format='%m-%d')

In [46]:

list = ['date','Origin_Airport','Destination_Airport','Airline','Carrier_Code(IATA)','Tail_Number']
for name in list:
    df[name] = encoder.fit_transform(df[name])

df["Estimated_Departure_Time"].fillna(df["Estimated_Departure_Time"].mode()[0],inplace=True)
df["Estimated_Arrival_Time"].fillna(df["Estimated_Arrival_Time"].mode()[0],inplace=True)
# df["Carrier_ID(DOT)"].fillna(df["Carrier_ID(DOT)"].mode()[0],inplace=True)


# df = df.astype({'Estimated_Departure_Time': 'int',"Estimated_Arrival_Time":'int',"Carrier_ID(DOT)":'int','Distance':'int'})

In [47]:
df_not_na = df.dropna(subset=['Delay'])
df_na = df[df['Delay'].isnull()]

In [48]:
scaler = StandardScaler()
droplist = ['Delay','ID','Month', 'Day_of_Month','Cancelled','Origin_Airport_ID', 'Origin_State', 'Diverted','Destination_Airport_ID', 'Destination_State','Carrier_ID(DOT)']
X = df_not_na.drop(droplist,axis=1)
y = df_not_na['Delay']
scaler.fit(X)
scaler.transform(X)
X2 = df_na.drop(droplist,axis=1)
y2 = df_na['Delay']
scaler.fit(X2)
scaler.transform(X2)

array([[-1.28930931,  0.70083555,  0.68329295, ...,  1.06614666,
         0.59931734, -0.85631607],
       [-1.01870098, -1.02375733,  0.72331999, ...,  0.57182627,
        -1.60918705,  0.29709119],
       [ 0.66293646,  0.51380779, -1.09791032, ..., -1.40545526,
        -1.70283777,  0.50508267],
       ...,
       [-0.90272599, -0.40164392,  0.64326591, ...,  1.06614666,
        -1.23238061, -0.15670839],
       [ 0.66873521,  0.55121334, -1.38810636, ..., -0.66397468,
         1.63939125,  0.70361998],
       [ 1.03019061,  1.00795483,  1.23366475, ..., -0.66397468,
         0.24013928, -1.04539923]])

In [49]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
rf_clf = RandomForestClassifier(random_state=221,max_depth=10)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('랜덤 포레스트 정확도: {:.4f}'.format(accuracy))

랜덤 포레스트 정확도: 0.8226


In [50]:
pred2 = rf_clf.predict(X2)
df_na['Delay'] = pred2

In [51]:
final_data = pd.concat([df_not_na,df_na])
final_data['Delay'] = final_data['Delay'].apply(np.int64)
final_data.to_csv('finaldata.csv',encoding='utf-8',index=False)